In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation
#'irage_dataset.csv'
dt = np.dtype(np.float32) 
df = pd.read_csv("irage_dataset.csv")
def test_train_data(filename):
    classifier_column = "fut_direction"
    # Limit input only to 100000 to test once
    temp_input = df[0:50000]       
    temp_input.fillna(value = -99999, inplace=True)
    temp_input_x = temp_input.drop(["date", "fut_spread", "fut_direction", "30secAhead", "1minAhead"], 1)
    temp_output_y_arr = np.array(temp_input[classifier_column])
    labels = []
    for i in range(len(temp_output_y_arr)):
        arr = [0, 0, 0, 0, 0]
        arr[int(temp_output_y_arr[i]) + 2 ] += 1
        labels.append(arr)
    temp_x = preprocessing.scale(temp_input_x)
    x_train = temp_x[1:10000]
    y_train = labels[1:10000]
    x_test = temp_x[10001:12000]
    y_test = labels[10001:12000]
    return np.array(x_train, dtype=dt), np.array(y_train, dtype=dt), np.array(x_test, dtype=dt), np.array(y_test,dtype=dt)
#     x_test = temp
#     return x_train, x_test, y_train, y_test 

# test_train_data("")

In [31]:
import numpy as np
import theano as theano
import theano.tensor as T
# from utils import *
import operator
from read_data import *
import time
#theano.scan(fn, sequences=None, outputs_info=None, non_sequences=None, n_steps=None, truncate_gradient=-1, go_backwards=False, mode=None, name=None, profile=False, allow_gc=None, strict=False, return_list=False)

#fn ==> fn is a function that describes the operations involved in one step of scan. 

#sequences ==> 

class RNNTheano:
    def __init__(self, input_dim, hidden_dim=500, output_dim=5, bptt_truncate=5):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.bptt_truncate = bptt_truncate
        self.hidden_dim = hidden_dim
        #randomly initialize the network parameters
        U = np.random.uniform(-np.sqrt(1/input_dim), np.sqrt(1/input_dim), (hidden_dim, input_dim))
        W = np.random.uniform(-np.sqrt(1/hidden_dim), np.sqrt(1/hidden_dim), (hidden_dim , hidden_dim))
        V= np.random.uniform(-np.sqrt(1/hidden_dim), np.sqrt(1/hidden_dim), ( output_dim, hidden_dim ))
        #converting variables into theano variables
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.fvector('x')
        y = T.ivector('y')
        def forward_prop_step(x_t, s_t_prev, U, V, W):
            s_t = T.tanh(U.dot(x_t) + W.dot(s_t_prev))
            o_t = T.nnet.softmax(V.dot(s_t))
            return [o_t[0], s_t]
        [o, s], updates = theano.scan(
            forward_prop_step,
            sequences = x,
            outputs_info=[None, dict(initial=T.zeros([self.hidden_dim, 1]))],
            non_sequences=[U,V,W],
            truncate_gradient=self.bptt_truncate,
            strict=True
            )
        
        prediction = T.argmax(o, axis = 1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        # Gradients
        dU = T.grad(o_error, U)
        dW = T.grad(o_error, W)
        dV = T.grad(o_error, V)
        
        # Functions
        self.forward_propagation = theano.function([x], o)
        self.predict = theano.function([x], prediction)
        self.ce_error = theano.function([x,y], o_error)
        self.bptt = theano.function([x, y], [dU, dV, dW])
        
        #SGD
        learning_rate = T.scalar('learning_rate')
        self.sgd_step = theano.function([x, y, learning_rate], [] , 
                                       updates = [
                                           (self.U, self.U - learning_rate * dU), 
                                           (self.V, self.V - learning_rate * dV),
                                           (self.W, self.W - learning_rate * dW)
                                       ])
        
        def calculate_total_loss(self, X, Y):
            return np.sum([self.ce_error(x, y) for x, y in zip(X, Y)])
        
        def calculate_loss(self, X, Y):
            
            #total number of outputs
            total_output = np.sum([len(y) for y in Y])
            return self.calculate_total_loss(X, Y)/float(total_output)
        

def softmax(x):
    earr = np.exp(x - np.max(x))
    return earr/np.sum(earr)

def save_model_parameters(output_file, tmodel):
    u, v, w = tmodel.U.get_value(), tmodel.V.get_value(), tmodel.W.get_value()
    np.savez(output_file, U = u, V = v , W = w)
    print "saved model parameter at : %s in file "%(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"), output_file)
    

def train_with_sgd(model, x_train, y_train, learning_rate=0.005, nepoch=1, evaluate_loss_after=1):
    losses = []
    example_seen = 0
    for epoch in range(nepoch):
        if(epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(x_train, y_train)
            losses.append((example_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
            print "%s: loss after example_seen=%d, epoch=%d, :: %f"%(time, example_seen, epoch, loss)
            #adjust the learning rate if loss increases
            if (len(losses)>1 and losses[-1][1] > losses[-2][1]):
                learning_rate = 0.5 * learning_rate
                print "New learning rate :: %f"%learning_rate
            save_model_parameters("saved_theano_parameters",model)
        
        for i in range(len(y_train)):
            #one sgd step 
            model.sgd_step(x_train[i], y_train[i], learning_rate)
            example_seen += 1

train_x, test_x, train_y, test_y = test_train_data("irage_dataset.csv")

model = RNNTheano(input_dim = 142, hidden_dim=200, output_dim= 5)
t1= time.time()
learning_rate = 0.005
print len(train_x[1])
print len(train_y[1])
print model.hidden_dim
print model.input_dim
print model.output_dim
print train_x[1]
model.sgd_step(train_x[1], train_y[1], learning_rate)
t2 = time.time()
print ("SGD step time : %f Milliseconds" %((t2-t1)/1000.0))

train_with_sgd(model, train_x, train_y, nepoch = 4)